In [ ]:
using QuadGK, PyPlot, DelimitedFiles

In [ ]:
function xi(eta,u)
    q = clamp((1.0+eta)/(1.0-eta),-1e80,1e80)
    return (log(q)+u*1im)/(eta+tan(u/2)*1im)    
end

In [ ]:
function phi_pt1(r,t)
    eta = r/t
    q = clamp((1.0+eta)/(1.0-eta),-1e80,1e80)
    return exp(-t)/(4*pi*r*t^2)*t*log(q)
end

In [ ]:
function integrand_pt(u,eta,t)
    return sec(u/2)^2*real((eta+1im*tan(u/2))*xi(eta,u)^3*exp(t/2*(1-eta^2)*xi(eta,u)))
end

In [ ]:
function phi_pt(r,t)
    r = clamp(r,1e-10,1e80)
    eta = r/t
    g = 0.0
    if eta<1.0
        g,error = quadgk(x->integrand_pt(x,eta,t),0,pi,atol = 1e-2)
    end
    f = 1/(2*pi)*exp(-t) /(4*pi*r*t^2)*(t/2)^2*(1-eta^2)*g
    f = f + phi_pt1(r,t)
    return f
end

In [ ]:
function phi_l(t,r)
    eta = r/t
    if eta<1.0
        f,error = quadgk(w->phi_pt(t*sqrt(eta^2+w^2),t),0,sqrt(1-eta^2),atol = 1e-5)
        phi_l0 = exp(-t)/(2*pi*t^2) / sqrt(1-eta^2)
        return phi_l0 + (2*t)*f
    else
        return 0.0
    end
end

In [ ]:
function integrand(tau,r,sigmas,sigmaa)
   return  sigmas*exp(-(sigmaa+sigmas)*tau)*phi_l(tau,r-tau)
end

In [ ]:
function getflux(t,x,y,omegax,omegay,sigmas,sigmaa)
    radii = sqrt.((x .- omegax*t).^2 .+ (y .- omegay*t).^2)
    return phi_l.(1.0 .- t,radii) 
end

In [ ]:
function correctintegrand(t,x,y,omegax,omegay,sigmas,sigmaa)
    return exp.(-(sigmas+sigmaa).*t) .*getflux.(t,x,y,omegax,omegay,sigmas,sigmaa)
end

In [ ]:
function assemble(x,y,quadpoints)
    sigmaa = 0.0
    sigmas = 1.0
    flux = zeros(size(quadpoints)[1])
    for i=1:size(quadpoints)[1]
        omegax = quadpoints[i,1]
        omegay = quadpoints[i,2]
        #fl =getflux(t,x,y,omegax,omegay,sigmas,sigmaa);
        #flux[i] = (sum(fl .* exp.(-(sigmas+sigmaa).*t))/length(t))
        function myfun(tau)
            return correctintegrand(tau,x,y,omegax,omegay,sigmas,sigmaa)
        end
        dummy = (myfun(0.0))
        # instead of integrating to 1, see where we can cut the integral, maxbe this helps quadgk to not suck so hard

        flux[i],_ = quadgk(myfun,0,1,maxevals=200)
        #println(flux[i])
        #println(i)
    end
    return flux
end

In [ ]:
quadpoints = readdlm("../points.txt");
quadweights = readdlm("../weights.txt")
x = 0.5
y = 0.5
t = 0.3
print(correctintegrand(t,x,y,0.3,0.3,1.0,1.0))
@time z = assemble(x,y,quadpoints);

In [ ]:
nx = 100
ny = 100
nq = 36
X = range(0,stop=1.5,length=nx)
Y = range(0,stop=1.5,length=ny)
T =range(0,stop=1,length=10)

phi = zeros(nx,ny)
psi = zeros(nq,nx,ny)
for i=1:nx
    print(i)
    print(", ")
    for j=1:ny
        p = assemble(X[i],Y[j],quadpoints)
        phi[i,j] = sum(p .* quadweights)
        psi[:,i,j] = p 
    end
end

In [ ]:
#oldphi = deepcopy(phi)
PyPlot.pcolormesh(phi/4/pi,vmin = 0,vmax = 0.4)
#savephi = deepcopy(phi)

In [ ]:
sort(phi/4/pi)

In [ ]:
phi